In [ ]:
%pip install -q ipywidgets matplotlib
%matplotlib inline

In [ ]:
import pandas as pd
from ipywidgets import interact
import matplotlib.pyplot as plt

In [ ]:
# load data
pd.set_option('display.max_rows', 20)
stockCodes = ["AMZN", "APPL", "META", "NFLX"]
stocks_df = pd.DataFrame()

# format the data correctly
for stockCode in stockCodes:
    df = pd.read_csv(f"Resources/{stockCode}-HistoricalData.csv")
    df = df.rename(columns={'Close/Last': 'Close'})
    df['StockCode'] = stockCode
    df['Date'] = pd.to_datetime(df['Date'])
    df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
    df['Close'] = df['Close'].str.replace('$', '')
    df['Open'] = df['Open'].str.replace('$', '')
    df['High'] = df['High'].str.replace('$', '')
    df['Low'] = df['Low'].str.replace('$', '')
    df = df.convert_dtypes()
    #display(df)
    stocks_df = pd.concat([stocks_df,df])

# convert the data types to be float for the numbers
stocks_df[['Close','Open','High','Low']] = stocks_df[['Close','Open','High','Low']].astype(float)
# convert the data to be correct
stocks_df['Date'] = pd.to_datetime(stocks_df['Date'], format='%Y-%m-%d')

#display(stocks_df.dtypes)
#display(stocks_df)

# save the CSV in one file
#stocks_df.to_csv(f"Resources/clean-HisotricalData.csv",mode = 'w', index=False)

In [ ]:
# get the average and variance of the monthly stock sample
stocks = dict()
stocks_monthly_df = None
stocks_monthly_df = stocks_df.copy()
stocks_monthly_df = stocks_monthly_df.set_index('Date')
stocks_monthly_df = stocks_monthly_df.sort_index()
for stockCode in stockCodes: 
    stocks[stockCode+'_mean'] = stocks_monthly_df.loc[stocks_monthly_df['StockCode'] == stockCode, 'Close':'Low'].resample('M').mean()
    stocks[stockCode+'_var']  = stocks_monthly_df.loc[stocks_monthly_df['StockCode'] == stockCode, 'Close':'Low'].resample('M').var()

In [ ]:
#plot the graph
def display_plot(valuation, statistical_method, current_stock):
    fig, ax = plt.subplots(figsize=(14,6))
    plt.title(f"{statistical_method} for {valuation} price vs Volume for {current_stock}")

    if statistical_method=='Variance':
        statistical_method = 'var'
    elif statistical_method=='Average':
        statistical_method = 'mean'

    stocks[current_stock+'_'+statistical_method].plot(y=[valuation], ax = ax, ylabel=valuation)
    stocks[current_stock+'_'+statistical_method].plot(y=['Volume'], ax = ax, secondary_y=True, ylabel='Volume')

    plt.show()

# make the graph interactive
interact(display_plot, 
         valuation=[
             'High',
             'Low',
             'Open',
             'Close',
         ],
         statistical_method = [
            'Variance',
            'Average',
         ],
         current_stock=[
             ('Amazon', 'AMZN'),
             ('Apple', 'APPL'),
             ('Netflix', 'NFLX'),
             ('Meta', 'META'),
         ]);